# Importing Necessary Libraries

In [2]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, LSTM, Bidirectional,Conv1D,MaxPooling1D,Flatten
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

# Importing Dataset

In [3]:
data_1 = pd.read_json("Sarcasm_Headlines_Dataset.json", lines=True)
data_2 = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines=True)
data =  pd.concat([data_1, data_2])
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


# Text Cleaning & Preprocessing

In [4]:
def clean_text(text):
    text = text.lower()
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    emoji = re.compile("["
                           u"\U0001F600-\U0001FFFF"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    
    text = emoji.sub(r'', text)
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)        
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text) 
    text = re.sub(r"\'ll", " will", text)  
    text = re.sub(r"\'ve", " have", text)  
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"did't", "did not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"couldn't", "could not", text)
    text = re.sub(r"have't", "have not", text)
    text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-]", "", text)
    return text

In [5]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def CleanTokenize(df):
    head_lines = list()
    lines = df["headline"].values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        head_lines.append(words)
    return head_lines

head_lines = CleanTokenize(data)
head_lines[0:10]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['former',
  'versace',
  'store',
  'clerk',
  'sues',
  'secret',
  'black',
  'code',
  'minority',
  'shoppers'],
 ['roseanne',
  'revival',
  'catches',
  'thorny',
  'political',
  'mood',
  'better',
  'worse'],
 ['mom',
  'starting',
  'fear',
  'sons',
  'web',
  'series',
  'closest',
  'thing',
  'grandchild'],
 ['boehner',
  'wants',
  'wife',
  'listen',
  'come',
  'alternative',
  'debtreduction',
  'ideas'],
 ['jk', 'rowling', 'wishes', 'snape', 'happy', 'birthday', 'magical', 'way'],
 ['advancing', 'worlds', 'women'],
 ['fascinating', 'case', 'eating', 'labgrown', 'meat'],
 ['ceo', 'send', 'kids', 'school', 'work', 'company'],
 ['top', 'snake', 'handler', 'leaves', 'sinking', 'huckabee', 'campaign'],
 ['fridays', 'morning', 'email', 'inside', 'trumps', 'presser', 'ages']]

In [8]:
all_lengths = [len(x) for x in head_lines]
max_length_sentence = max(all_lengths)
max_length_sentence

106

# Train_Test_Split

In [9]:
validation_split = 0.2
max_length = max_length_sentence


tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(head_lines)
sequences = tokenizer_obj.texts_to_sequences(head_lines)

word_index = tokenizer_obj.word_index
print("unique tokens - ",len(word_index))
vocab_size = len(tokenizer_obj.word_index) + 1
print('vocab size -', vocab_size)

lines_pad = pad_sequences(sequences, maxlen=max_length, padding='post')
sentiment =  data['is_sarcastic'].values

indices = np.arange(lines_pad.shape[0])
np.random.shuffle(indices)
lines_pad = lines_pad[indices]
sentiment = sentiment[indices]

num_validation_samples = int(validation_split * lines_pad.shape[0])

X_train_pad = lines_pad[:-num_validation_samples]
y_train = sentiment[:-num_validation_samples]
X_test_pad = lines_pad[-num_validation_samples:]
y_test = sentiment[-num_validation_samples:]

unique tokens -  28657
vocab size - 28658


In [10]:
print('Shape of X_train_pad:', X_train_pad.shape)
print('Shape of y_train:', y_train.shape)

print('Shape of X_test_pad:', X_test_pad.shape)
print('Shape of y_test:', y_test.shape)

Shape of X_train_pad: (44263, 106)
Shape of y_train: (44263,)
Shape of X_test_pad: (11065, 106)
Shape of y_test: (11065,)


# Importing Dataset from Kaggle

In [ ]:
# ! pip install kaggle
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json #Your kaggle.json file

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
# ! kaggle datasets download -d bertcarremans/glovetwitter27b100dtxt

 96% 382M/397M [00:03<00:00, 145MB/s]
100% 397M/397M [00:03<00:00, 126MB/s]


In [ ]:
# ! unzip glovetwitter27b100dtxt.zip

Archive:  glovetwitter27b100dtxt.zip
  inflating: glove.twitter.27B.100d.txt  


# Loading GloVe 

In [6]:
# embeddings_index = {}
# embedding_dim = 100
# GLOVE_DIR = ""
# f = open(os.path.join(GLOVE_DIR, 'glove.twitte+r.27B.100d.txt'), encoding = "utf-8")
# for line in f:
#     values = line.split()
#     word = values[0]
#     coefs = np.asarray(values[1:], dtype='float32')
#     embeddings_index[word] = coefs
# f.close()

# print('Found %s word vectors.' % len(embeddings_index))

#Load Already Saved GloVe Model from Local
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format("glove.twitter.27B.100d.txt.word2vec",binary = False)

# Creating Embedding Matrix + Layer

In [11]:
len(word_index)

28657

In [12]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
c = 0
for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
    try:
        embedding_vector = model.get_vector(word)
    except:
        embedding_vector = None
    if embedding_vector is not None:
        c+=1
        embedding_matrix[i] = embedding_vector
print(c)

24755


In [13]:
import pickle as pkl
with open("EmbeddingMatrixGloVe.pkl",'wb') as f:
    pkl.dump([embedding_dim,embedding_matrix],f)

In [13]:
embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

# Creating the main Neural Network with LSTM

In [14]:
model = Sequential()
model.add(embedding_layer)
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.25))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(model.summary())

Summary of the built model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 100)           2865800   
                                                                 
 lstm (LSTM)                 (None, 64)                42240     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2,908,105
Trainable params: 42,305
Non-trainable params: 2,865,800
_________________________________________________________________
None


#Evaluating Model

In [15]:
history = model.fit(X_train_pad, y_train, batch_size=32, epochs=25, validation_data=(X_test_pad, y_test))

Epoch 1/25
1384/1384 [==============================] - 33s 22ms/step - loss: 0.5078 - accuracy: 0.7513 - val_loss: 0.4360 - val_accuracy: 0.7978
Epoch 2/25
1384/1384 [==============================] - 31s 22ms/step - loss: 0.4248 - accuracy: 0.8044 - val_loss: 0.3804 - val_accuracy: 0.8300
Epoch 3/25
1384/1384 [==============================] - 32s 23ms/step - loss: 0.3894 - accuracy: 0.8247 - val_loss: 0.3620 - val_accuracy: 0.8395
Epoch 4/25
1384/1384 [==============================] - 31s 23ms/step - loss: 0.3578 - accuracy: 0.8410 - val_loss: 0.3398 - val_accuracy: 0.8542
Epoch 5/25
1384/1384 [==============================] - 31s 22ms/step - loss: 0.3343 - accuracy: 0.8549 - val_loss: 0.3282 - val_accuracy: 0.8596
Epoch 6/25
1384/1384 [==============================] - 32s 23ms/step - loss: 0.3177 - accuracy: 0.8628 - val_loss: 0.3122 - val_accuracy: 0.8680
Epoch 7/25
1384/1384 [==============================] - 32s 23ms/step - loss: 0.2987 - accuracy: 0.8740 - val_loss: 0.2998 -

#Saving the model

In [16]:
model.save('my_model.h5')

# Making Predictions

In [17]:
def predict_sarcasm(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = model.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!" 
    else: return "It's not a sarcasm."

In [18]:
predict_sarcasm("I was depressed. He asked me to be happy. I am not depressed anymore.")

1/1 [==============================] - 0s 351ms/step


"It's a sarcasm!"

In [19]:
predict_sarcasm("You just broke my car window. Get out from here.")

1/1 [==============================] - 0s 64ms/step


"It's not a sarcasm."

In [20]:
predict_sarcasm("You just saved my dog's life. Sorry.")

1/1 [==============================] - 0s 36ms/step


"It's not a sarcasm."

In [21]:
predict_sarcasm("I want a million dollars!")

1/1 [==============================] - 0s 34ms/step


"It's not a sarcasm."

In [22]:
predict_sarcasm("I just won a million dollars!")

1/1 [==============================] - 0s 27ms/step


"It's a sarcasm!"

In [23]:
model.evaluate(X_test_pad, y_test)

346/346 [==============================] - 3s 8ms/step - loss: 0.2065 - accuracy: 0.9260


[0.20645006000995636, 0.9259828329086304]

# Training the model with Embedding + CNN 1D

In [31]:
cnn = Sequential()
cnn.add(embedding_layer)
cnn.add(Conv1D(filters=32,kernel_size=8,activation='relu'))
cnn.add(MaxPooling1D(pool_size=2))
cnn.add(Flatten())
cnn.add(Dense(100, activation='relu'))
cnn.add(Dense(1, activation='sigmoid'))

cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print('Summary of the built model...')
print(cnn.summary())

Summary of the built model...
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 100)           2865800   
                                                                 
 conv1d_2 (Conv1D)           (None, 18, 32)            25632     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 9, 32)            0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 288)               0         
                                                                 
 dense_5 (Dense)             (None, 100)               28900     
                                                                 
 dense_6 (Dense)             (None, 1)                 101       
                        

In [32]:
cnn.fit(X_train_pad, y_train, batch_size=32, epochs=25, validation_data=(X_test_pad, y_test))

Epoch 1/25
1384/1384 [==============================] - 15s 11ms/step - loss: 0.4677 - accuracy: 0.7760 - val_loss: 0.3985 - val_accuracy: 0.8217
Epoch 2/25
1384/1384 [==============================] - 17s 12ms/step - loss: 0.3249 - accuracy: 0.8611 - val_loss: 0.3335 - val_accuracy: 0.8558
Epoch 3/25
1384/1384 [==============================] - 17s 12ms/step - loss: 0.2239 - accuracy: 0.9108 - val_loss: 0.3030 - val_accuracy: 0.8787
Epoch 4/25
1384/1384 [==============================] - 16s 12ms/step - loss: 0.1562 - accuracy: 0.9413 - val_loss: 0.2728 - val_accuracy: 0.9096
Epoch 5/25
1384/1384 [==============================] - 17s 12ms/step - loss: 0.1108 - accuracy: 0.9604 - val_loss: 0.2664 - val_accuracy: 0.9195
Epoch 6/25
1384/1384 [==============================] - 13s 9ms/step - loss: 0.0841 - accuracy: 0.9707 - val_loss: 0.3185 - val_accuracy: 0.9172
Epoch 7/25
1384/1384 [==============================] - 16s 12ms/step - loss: 0.0670 - accuracy: 0.9763 - val_loss: 0.3187 - 

In [33]:
def predict_sarcasm_cnn(s):
    x_final = pd.DataFrame({"headline":[s]})
    test_lines = CleanTokenize(x_final)
    test_sequences = tokenizer_obj.texts_to_sequences(test_lines)
    test_review_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')
    pred = cnn.predict(test_review_pad)
    pred*=100
    if pred[0][0]>=50: return "It's a sarcasm!" 
    else: return "It's not a sarcasm."

In [37]:
predict_sarcasm_cnn("You just saved my dog's life.Sorry.")

1/1 [==============================] - 0s 93ms/step


"It's not a sarcasm."

In [38]:
predict_sarcasm_cnn("I don't know your name, Andrew?")

1/1 [==============================] - 0s 37ms/step


"It's not a sarcasm."

In [48]:
predict_sarcasm_cnn("You just broke my car window. Great")

1/1 [==============================] - 0s 25ms/step


"It's not a sarcasm."